In [1]:
#Importamos las bibliotecas que usaremos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LinearRegression

data = pd.read_csv('data/dataset_turnos.csv')
data.sample(3)



,Unnamed: 0,appointmentfromtime,DayName,priordayholiday,nextdayholiday,day,hour,outofschedule,appointmentstate,age,daysbeforereserved,speciality,gender,citypatient,healthcoverage,buildingcity,firsttime,emailvalid,endofmonth
21658,29474,10/6/2021 15:40,Thursday,0,0,10,15,True,1,41,0,Infectología,F,Mar del Plata,Con OS,Mar del plata,1,1,0
47799,65092,29/11/2021 17:20,Monday,0,0,29,17,False,1,1,14,Pediatría,F,Mar del Plata,Con OS,Mar del plata,0,1,1
32570,44244,26/8/2021 10:20,Thursday,0,0,26,10,False,0,11,18,Ortopedia y traumatología,F,Mar del Plata,Sin OS,Mar del plata,1,1,0


In [2]:

x_columnas = ['endofmonth','emailvalid','DayName','priordayholiday','nextdayholiday','day','hour','age','daysbeforereserved','outofschedule','firsttime','speciality','gender','citypatient','healthcoverage','buildingcity']
y_columnas = ['appointmentstate']
data_modelo =  pd.concat([data[y_columnas],data[x_columnas]], axis = 1)
data_modelo

,appointmentstate,endofmonth,emailvalid,DayName,priordayholiday,nextdayholiday,day,hour,age,daysbeforereserved,outofschedule,firsttime,speciality,gender,citypatient,healthcoverage,buildingcity
0,0,0,0,Monday,0,0,4,8,1,0,False,0,Pediatría,F,Mar del Plata,Con OS,Mar del plata
1,1,0,1,Monday,0,0,4,8,2,0,False,1,Pediatría,F,Mar del Plata,Con OS,Mar del plata
2,1,0,1,Monday,0,0,4,9,11,18,False,1,Pediatría,F,Mar del Plata,Con OS,Mar del plata
3,1,0,1,Monday,0,0,4,9,2,0,False,1,Pediatría,M,Mar del Plata,Con OS,Mar del plata
4,1,0,0,Monday,0,0,4,9,1,18,False,1,Pediatría,M,Mar del Plata,Con OS,Mar del plata
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83134,1,0,1,Monday,0,0,18,18,8,48,False,0,Pediatría,M,Mar del Plata,Con OS,Mar del plata
83135,1,0,1,Monday,0,0,18,18,45,38,False,1,Clínica Médica,F,Mar del Plata,Con OS,Mar del plata
83136,1,0,1,Monday,0,0,18,18,1,20,False,0,Pediatría,M,Mar del Plata,Con OS,Mar del plata
83137,1,0,1,Monday,0,0,18,18,4,16,False,0,Pediatría,M,Mar del Plata,Con OS,Mar del plata


In [ ]:
# visualizamos la matriz de correlación en Seaborn usando a heatmap
sns.set(rc = {'figure.figsize':(15,8)})
sns.heatmap(data_modelo.corr(), vmin=-1, vmax=1, center=0, annot=True);

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(18,12))
fig.suptitle('Histogramas normalizados')
for c, ax in zip(data_modelo.columns[:-1], axes.flatten()):
    sns.histplot(data = data_modelo.loc[data_modelo['appointmentstate']==0, c].dropna(), stat = 'density', ax = ax, kde = False )
    sns.histplot(data = data_modelo.loc[data_modelo['appointmentstate']==1, c].dropna(), stat = 'density', kde=False, ax=ax, color = 'orange')
    ax.legend(['Ausente', 'Presente'])

In [ ]:
data_dummies=pd.get_dummies(data_modelo[['DayName','speciality','gender','healthcoverage']], drop_first = True)
data_dummies

In [ ]:

data_modelo = pd.concat([data[['endofmonth','day','hour','emailvalid','appointmentstate','priordayholiday','nextdayholiday','age','daysbeforereserved','outofschedule','firsttime']],data_dummies], axis = 1)
data_modelo = data_modelo.reset_index(drop=True)
data_modelo.head(3)

In [ ]:
# visualizamos la matriz de correlación en Seaborn usando a heatmap
plt.figure(figsize=(8,22))
sns.heatmap(data_modelo.corr()[['appointmentstate']], annot=True)

In [ ]:
#Genero mi variable X
X=data_modelo.drop(['appointmentstate'],axis=1)

#Genero la variable y
y = data_modelo[y_columnas].squeeze()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 0)

In [ ]:
y_train.mean(), y_test.mean()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_std, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(y_train, lr.predict(X_train_std))

In [ ]:
sns.heatmap(confusion_matrix(y_train, lr.predict(X_train_std)), annot=True, fmt='4d')
plt.xlabel('Predichos')
plt.ylabel('Reales')

In [ ]:
lr.predict_proba(X_train_std[y_train==0])[:,1]

In [ ]:
# Histograma de probabilidades
sns.histplot(data = lr.predict_proba(X_train_std[y_train==0])[:,1], stat = 'density')
sns.histplot(data = lr.predict_proba(X_train_std[y_train==1])[:,1], stat = 'density', color = 'orange')
ylim = plt.ylim()
plt.vlines(0.5, ylim[0], ylim[1])
plt.ylim(ylim)
plt.legend(['Umbral', 'Ausentes', 'Presentes'])

In [ ]:
umbral = 0.8
probs = lr.predict_proba(X_train_std)[:,1]
y_pred_train = probs > umbral

In [ ]:
accuracy_score(y_train, y_pred_train)

In [ ]:
sns.heatmap(confusion_matrix(y_train, y_pred_train), annot=True, fmt='4d')
plt.xlabel('Predichos')
plt.ylabel('Reales')